In [1]:
from copy import deepcopy
from operator import itemgetter

# defining the Literal class
class Literal:
    
    # consists of the name and its sign
    def __init__(self, name, sign=True):
        self.name = name
        self.sign = sign
        
    # for when comparing the literal in the clause with the literal in the model
    # equal regardless of sign
    def __eq__(self, other):
        return other.name == self.name
    
    # allows negative literals to be defined in clauses
    def __neg__(self):
        return Literal(self.name, sign = not self.sign)
    
    def __str__(self):
        # if the sign is true, just return the name
        if self.sign:
            return self.name
        # if sign is not true, return the name with a negative sign
        return "-" + self.name
    
    # need to make hashable
    def __hash__(self):
        return hash(self.name)
      
SAT_model = {} # dictionary that will store the satisfiable model. Global variable
set_true = Literal("true") # what will be assigned to sets that are true

In [2]:
def DPLL(clauses, symbols, model):
    
    # this will go through the clauses and literals in the actual KB
    for clause in clauses:
        
        to_remove = set()
        
        for literal in clause:
            if literal.name in model:
                # For the case if literal is true
                # which is when the literal sign matches the literal in the model
                if (literal.sign and model[literal.name]) or (not literal.sign and not model[literal.name]):
                    clause.add(set_true)
                    break
                
                # or if literal is false and there are more literals.
                # remove the literal (this is because anything or false just the anything, as discussed in class)
                elif len(clause) > 1:
                    to_remove.add(literal)
                
                # if the literal that is found in the model is the only one in the clause, and it was false
                # return False, because the clause failed (all the literals in the clause are false)
                else:
                    return False
        
        # will remove the literals in the clause that are false
        clause -= to_remove
    
    # if all clauses have Literal("true") inside 
    # (meaning all the clauses in the KB are true/in the model)            
    if all ([set_true in clause for clause in clauses]):
        
        # use global to get to the SAT_model variable that is outside the function
        global SAT_model
        
        # the inputted model is satisfiable, so save it to the global variable
        SAT_model = model
        return True

    if symbols:
        # get the first symbol
        p = symbols.pop(0)
        
        # make two models that will have either have that symbol assigned to true or false
        true_model = deepcopy(model)
        false_model = deepcopy(model)
        
        # get the literal at p[1]. p[0] is the frequency.
        true_model[p[1].name] = True
        false_model[p[1].name] = False
        
        #return the model that is true
        return DPLL(deepcopy(clauses), deepcopy(symbols), true_model) or \
               DPLL(deepcopy(clauses), deepcopy(symbols), false_model)
    
    # if all has failed, then return False
    # could not prove that model was satisfiable :(
    return False


def DPLLSatisfiable(KB):
    
    # make a dictionary for the symbols in the KB
    symbols = {}
    for clause in KB:
        for symbol in clause:
            
            # if the symbol is already there, we just add one
            if symbol in symbols:
                symbols[symbol] += 1
                
            # if the symbols isn't in the dictionary yet,
            # assign 1 to that symbol
            else:
                symbols[symbol] = 1
                
    # sort so that most frequent symbols are first aka degree heuristic
    # most frequent symbols will be selected/popped first
    symbols = sorted(([degree, key] for key,degree in symbols.items()), 
                     key = lambda x:x[0], reverse = True)

    # get either true or false for satisfiability of the KB
    SAT = DPLL(KB, list(symbols), {})
    
    # To return model if satisfiable
    if SAT:
        
        set_KB = set()
        
        # .union(*_) is used to make a set of the elements in the kb
        for symbol in set_KB.union(*KB):
            
            # symbols that weren't specified in the model are free
            if symbol.name not in SAT_model:
                SAT_model[symbol.name] = "Free"
                
            # if in model, it should just be the true or false it is assigned,
            # but as a string, not boolean
            else:
                SAT_model[symbol.name] = str(SAT_model[symbol.name])
            
        
    return SAT, SAT_model

In [3]:
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

KB = [{-A,B,E},{-B,A}, {-E,A}, {-E,D}, {-C,-F,-B}, {-E,B}, {-B,F}, {-B,C}]

DPLLSatisfiable(KB)


(True,
 {'A': 'False',
  'B': 'False',
  'C': 'Free',
  'D': 'Free',
  'E': 'False',
  'F': 'Free'})